In [29]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# sign- tells whether the forward or backward CN is to be returned:
def returnCNPropagator1(n, lam, sign=1):
    # Create Crank-Nicolson matrices (A^+ and A^-)
    A_plus = np.zeros((n, n))
    A_minus = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                A_plus[i, j] = (1 - lam)
                A_minus[i, j] = (1 + lam)
            elif abs(i - j) == 1:
                A_plus[i, j] = lam / 2
                A_minus[i, j] = -lam / 2
                if i==0 or i==n-1:
                    A_plus[i, j] = lam
                    A_minus[i, j] = -lam
    
    if sign == 1:
        return A_plus
    elif sign == -1:
        return A_minus

# returning the B_matrix, containing the constant term as a result of the VON NEUMANN BCs
def returnBmatrix(lam, dx, Cprime_start, Cprime_end, N, method = "euler"):
    B = np.zeros(N)
    
    if method == "euler":
        B[0] =  -2 * lam * dx * Cprime_start
        B[-1] = 2 * lam * dx * Cprime_end
    elif method== "cn":
        B[0] =  -1 * lam * dx * Cprime_start
        B[-1] = 1 * lam * dx * Cprime_end
        
    return B
    




# defining the parameters
D = 0.01
lam = 0.5
dx = 0.01

x_start = 0 # spatial grid 
x_end = 1 
x = np.arange(x_start,x_end + dx,dx)

N = len(x) 

# defining the initial Gaussian Profile, centred arsound xpeak
x_peak = 0.25
sigma = 0.1
C0 = []
for x_el in x:
    C0.append(np.exp(- (x_el-x_peak)**2 / sigma **2 ))


dt = lam * dx**2 / D # time step, value of Lambda will affect stability
t = np.arange(0, 1+dt, dt)

# Creating the matrices for the actual propagation
Aplus = returnCNPropagator1(N, lam , sign=1)
Aminus = returnCNPropagator1(N, lam, sign=-1)

B = returnBmatrix(lam, dx, 0, 0, N, method="cn")


# defining the 2d matrix to hold the entire concentration array for each time step. The first time step is already given.
C = np.zeros((len(t), len(x))) 

C[0] = C0.copy()

for i in range(1, len(t)):
    Cn = C[i-1] # from the last step
    C_intermediate = np.dot(Aplus, Cn) + B
    Cn_plus_1 = np.linalg.solve(Aminus, C_intermediate + B)
    
    C[i] = Cn_plus_1

In [30]:
pdffile = "cn-diffusion.pdf"

with PdfPages(pdffile) as pdf:
    for i in range(0, len(t)):
        # creating plot
        plt.plot(x,C[0], label=f"at time step = {round(t[0],5)}")
        plt.plot(x,C[i], label=f"at time step = {round(t[i],5)}")
        plt.legend()
        plt.xlabel("x")
        plt.ylabel("C(x,t)")
        plt.ylim(0,1)
        plt.title(f"Crank-Nicolson: dx={dx}, lam={lam}, D={D}, iter = {len(t)-1}")
        
        # exporting to the pdf.
        pdf.savefig()
        plt.close()